This notebook demonstrates how to leverage transfer learning to use your own image dataset to build and train an image classification model using MXNet and Amazon SageMaker.

We use, as an example, the creation of a trash classification model which, given some image, classifies it into one of three classes: compost, landfill, recycle. This is based on the [Show Before You Throw](https://www.youtube.com/watch?v=Ut1VGG6TOOw) project from an AWS DeepLens hackathon and the [Smart Recycle Arm](https://www.youtube.com/watch?v=QF0QjRjBwFs) project presented at the AWS Public Sector Summit 2019

---
1. [Prerequisites](#Prerequisites)
1. [Download Data](#Download-data)
1. [Fine-tuning the Image Classification Model](#Fine-tuning-the-Image-classification-model)
1. [Start the Training](#Start-the-training)
1. [Test your Model](#Inference)
1. [Deploy your Model to AWS DeepLens](#Deploy)

## Prequisites

- Amazon Sagemaker notebook should have internet access to download images needed for testing this notebook. This is turned ON by default. To explore another options review this link : [Sagemaker routing options](https://aws.amazon.com/blogs/machine-learning/understanding-amazon-sagemaker-notebook-instance-networking-configurations-and-advanced-routing-options/)
- The IAM role assigned to this notebook should have permissions to create a bucket (if it does not exist)
  - [IAM role for Amazon Sagemaker](https://docs.aws.amazon.com/glue/latest/dg/create-an-iam-role-sagemaker-notebook.html)
  - [S3 create bucket permissions](https://docs.aws.amazon.com/AmazonS3/latest/dev/using-with-s3-actions.html#using-with-s3-actions-related-to-buckets)

### Permissions and environment variables

Here we set up the linkage and authentication to AWS services. There are 2 parts to this:

* The roles used to give learning and hosting access to your data. This will automatically be obtained from the role used to start the notebook
* The Amazon sagemaker image classification docker image which need not be changed

In [3]:
import os
import urllib.request
import boto3, botocore

import sagemaker
from sagemaker import get_execution_role

import mxnet as mx
mxnet_path = mx.__file__[ : mx.__file__.rfind('/')]
print(f"mxnet_path: {mxnet_path}")

role = get_execution_role()
print(f"aws iam role: {role}")

sess = sagemaker.Session()

mxnet_path: /home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet
aws iam role: arn:aws:iam::677092314568:role/service-role/AmazonSageMaker-ExecutionRole-20210806T182472


## Amazon S3 bucket info
Enter your Amazon S3 Bucket name where your data will be stored, make sure that your SageMaker notebook has access to this S3 Bucket by granting `S3FullAccess` in the SageMaker role attached to this instance. See [here](https://docs.aws.amazon.com/sagemaker/latest/dg/gs-config-permissions.html) for more info.

DeepLens-compatible buckets must start with `deeplens`

In [29]:
# --> Bucket Exist and have permisions
BUCKET = 'deeplens-sampleproject-677092314568'

# --> Bucket doesn't exists
#BUCKET = 'deeplens-sampleproject-6770923145689'

# --> Bucket exists and doesn't have permisions
#BUCKET = 'ecommerce-terraforms'


PREFIX = 'deeplens-trash-test'

In [8]:
from sagemaker.amazon.amazon_estimator import get_image_uri
training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")
print(f"training_image: {training_image}")

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


training_image: 811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:1


We are going to check if we have the right bucket and if we have the right permissions.

Please make sure that the result from this cell is "Bucket access is Ok"

In [34]:
test_data = 'TestData'
s3 = boto3.resource('s3')
object = s3.Object(BUCKET, PREFIX+"/text.txt")

try:
    object.put(Body=test_data)
except botocore.exceptions.ClientError as e:
    print(e.response)
    if e.response['Error']['Code'] == 'AccessDenied':
        #cannot write on the bucket
        print(f"Bucket {BUCKET} is not writeble, make sure you have the right permissions")
    else:
        if e.response['Error']['Code'] == 'NoSuchBucket':
            #Bucket does not exist
            print(f"Bucket {BUCKET} does not exist")
        else:
            print("Shomething went wrong and I don't know what and/or why")
            raise
    
else:
    print("Bucket access is Ok")
    object.delete()

Bucket access is Ok


# Prepare data

It is assumed that your custom dataset's images are present in an S3 bucket and that different classes are separated by named folders, as shown in the following directory structure:
```
|-deeplens-bucket
   |-deeplens-trash

    |-images
    
        |-Compost 
    
        |-Landfill
    
        |-Recycle
 ```

In [35]:
!wget https://deeplens-public.s3.amazonaws.com/samples/deeplens-trash/trash-images.zip

--2021-08-06 17:43:22--  https://deeplens-public.s3.amazonaws.com/samples/deeplens-trash/trash-images.zip
Resolving deeplens-public.s3.amazonaws.com (deeplens-public.s3.amazonaws.com)... 52.217.169.161
Connecting to deeplens-public.s3.amazonaws.com (deeplens-public.s3.amazonaws.com)|52.217.169.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 253515836 (242M) [application/zip]
Saving to: ‘trash-images.zip’

trash-images.zip    100%[===================>] 241.77M  68.6MB/s    in 3.7s    

2021-08-06 17:43:26 (65.1 MB/s) - ‘trash-images.zip’ saved [253515836/253515836]



In [37]:
!pwd

/home/ec2-user/SageMaker/deeplens-sampleproject


In [41]:
!ls -l

total 247588
-rw-rw-r-- 1 ec2-user ec2-user      9258 Aug  6 17:43 aws-deeplens-custom-trash-detector.ipynb
-rw-rw-r-- 1 ec2-user ec2-user 253515836 Mar 10  2020 trash-images.zip


In [42]:
!rm -rf data/ && mkdir -p data/images
!unzip -qq trash-images.zip -d data/images
!rm trash-images.zip
!ls -l

total 16
-rw-rw-r-- 1 ec2-user ec2-user 10067 Aug  6 17:45 aws-deeplens-custom-trash-detector.ipynb
drwxrwxr-x 3 ec2-user ec2-user  4096 Aug  6 17:47 data


In [44]:
!ls data/images

Compost  Landfill  Recycling


In [45]:
!ls -al data/images/

total 36
drwxrwxr-x 5 ec2-user ec2-user  4096 Aug  6 17:47 .
drwxrwxr-x 3 ec2-user ec2-user  4096 Aug  6 17:47 ..
drwxr-xr-x 2 ec2-user ec2-user 12288 Feb 24  2020 Compost
drwxr-xr-x 2 ec2-user ec2-user 12288 Feb 24  2020 Landfill
drwxr-xr-x 2 ec2-user ec2-user  4096 Feb 24  2020 Recycling


In [46]:
!ls -l data/images/Compost/ data/images/Landfill/ data/images/Recycling/

data/images/Compost/:
total 101052
-rw-r--r-- 1 ec2-user ec2-user 509529 Jul 29  2018 20180403-195616.jpg
-rw-r--r-- 1 ec2-user ec2-user 485929 Jul 29  2018 20180403-195643.jpg
-rw-r--r-- 1 ec2-user ec2-user 515320 Jul 29  2018 20180403-195651.jpg
-rw-r--r-- 1 ec2-user ec2-user 520909 Jul 29  2018 20180403-195701.jpg
-rw-r--r-- 1 ec2-user ec2-user 513095 Jul 29  2018 20180403-195711.jpg
-rw-r--r-- 1 ec2-user ec2-user 487846 Jul 29  2018 20180403-195729.jpg
-rw-r--r-- 1 ec2-user ec2-user 504845 Jul 29  2018 20180403-195736.jpg
-rw-r--r-- 1 ec2-user ec2-user 505385 Jul 29  2018 20180403-195746.jpg
-rw-r--r-- 1 ec2-user ec2-user 507942 Jul 29  2018 20180403-195757.jpg
-rw-r--r-- 1 ec2-user ec2-user 505568 Jul 29  2018 20180403-195810.jpg
-rw-r--r-- 1 ec2-user ec2-user 412947 Jul 29  2018 20180403-195833.jpg
-rw-r--r-- 1 ec2-user ec2-user 417571 Jul 29  2018 20180403-195840.jpg
-rw-r--r-- 1 ec2-user ec2-user 496045 Jul 29  2018 20180403-195848.jpg
-rw-r--r-- 1 ec2-user ec2-user 468145 Jul 